# HW 12, Daniil Glukhovskiy

## Problem 2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import qrpm_funcs as qf
#import training set
#pound sterling
seriesnames=['DEXUSUK']
cdates,ratematrix=qf.GetFREDMatrix(seriesnames,startdate = '2014-12-31', enddate='2018-12-31')

lgdates,difflgs=qf.levels_to_log_returns(cdates,ratematrix, [1])

#compute mean, variance
d=np.array(difflgs)
m=np.mean(d,axis=0)
var_0 = np.var(d)

#display the output
#vectors and matrices are in fractional units;
#    fraction*100=percent
#    fraction*10000=basis point
#    (fraction^2)*10000=percent^2
np.set_printoptions(precision=4)
print("From",lgdates[0],"to",lgdates[-1],"(",len(lgdates),"observations):")
print("\nMeans:",m*10000,"bps/day")
print("Sample Variance:", var_0)

From 2015-01-02 to 2018-12-31 ( 1000 observations):

Means: [-1.9931] bps/day
Sample Variance: 4.010681260827864e-05


In [2]:
#Fit GARCH(1,1)
[a,b,c] = qf.Garch11Fit([0.4,0.1,3.0], difflgs)

In [3]:
print("GARCH(1,1) model parameters (a,b,c) =", [a, b, c])

GARCH(1,1) model parameters (a,b,c) = [0.10722216415743795, 0.834996132072086, 2.397142232522668e-06]


In [4]:
#import data for simulation
seriesnames=['DEXUSUK']
cdates_val,ratematrix_val=qf.GetFREDMatrix(seriesnames,startdate = '2018-12-31', enddate='2019-12-31')

lgdates_val,difflgs_val=qf.levels_to_log_returns(cdates_val,ratematrix_val, [1])

#compute mean
d_val=np.array(difflgs_val)
m_val=np.mean(d_val,axis=0)

##get rid of 2d-array, i know that's bad code
for i in range(249):
    difflgs_val[i] = difflgs_val[i][0]

#display the output

np.set_printoptions(precision=4)
print("From",lgdates_val[0],"to",lgdates_val[-1],"(",len(lgdates_val),"observations):")
print("\nMeans:",m*10000,"bps/day")


From 2019-01-02 to 2019-12-31 ( 249 observations):

Means: [-1.9931] bps/day


In [5]:
#form variances using (8.57)
var = np.zeros(249)
var[0] = var_0
for i in range(1, len(var)):
    var[i] = c + b*(var[i-1]) + a*(difflgs_val[i-1])**2
std = np.sqrt(var)

In [6]:
#compute statistics of GARCH(1,1) simulation
from scipy.stats import skew, kurtosis
y_over_sigma= difflgs_val * 1/std
print("Variance of series y/sigma from GARCH(1,1) model:", np.var(y_over_sigma))
print("Excess kurtosis of series y/sigma from GARCH(1,1) model:", kurtosis(y_over_sigma))


Variance of series y/sigma from GARCH(1,1) model: 0.8665483950770695
Excess kurtosis of series y/sigma from GARCH(1,1) model: 1.0629686677677173


Variance is 0.87, close enough to 1. Kurtosis is 1.06, could be better.

## Problem 4

In [7]:
#parameters
alpha = 0.1
k = 1.2
theta = 0.16
rho = 0
dt = 1/252
T = 252
max_phi = (2*k*theta)**0.5

In [8]:
#one simulation for fixed volvol 
def sim(phi):
    v = np.zeros(T)
    v[0] = theta
    log_x = np.zeros(T)
    for i in range(1, T):
        dz = np.random.normal(0, dt**0.5)
        dbeta = np.random.normal(0, dt**0.5)
        if v[i-1] < 0:
            v[i] = k*(theta - v[i-1])*dt
        else:
            v[i] = k*(theta - v[i-1])*dt + dz*phi*(v[i-1])**0.5
        if v[i] < 0:
            log_x[i] = (alpha - v[i]/2)*dt
        else:
            log_x[i] = (alpha - v[i]/2)*dt + dbeta*(v[i])**0.5
    return(kurtosis(log_x))

In [9]:
#median kurtosis after 1000 simulations with each volvol
median_kurt = []
for volvol in np.arange(0, max_phi, 0.1):
    kurt_sim = np.zeros(1000)
    for i in range(1000):
        kurt_sim[i] = sim(volvol)
    median_kurt.append(np.median(kurt_sim))

In [10]:
for i in range(7):
    print('phi = ', i/10, "median excess kurtosis = ", median_kurt[i])

phi =  0.0 median excess kurtosis =  -0.02969227856914447
phi =  0.1 median excess kurtosis =  0.09808285364557112
phi =  0.2 median excess kurtosis =  0.4908454068353474
phi =  0.3 median excess kurtosis =  1.0312975629261216
phi =  0.4 median excess kurtosis =  1.5235582578213172
phi =  0.5 median excess kurtosis =  1.8999760504730983
phi =  0.6 median excess kurtosis =  2.3273912914332344


As volvol increases, kurtosis of log-returns increases, as expected. 